## Description

This is a notebook with a simplified setup, which should show pure DS & CT speculation behaviour over an external stETH timeseries



```bash
python generate-events.py \
  --start-date 2024-01-01 \
  --end-date 2024-10-26 \
  --threshold 0.001 \
  --output events.json \
  --token-symbol stETH \
  --coin-gecko-id staked-ether \
  --vs-currency eth
```

In [1]:
# Install required packages if not already installed
# Uncomment the lines below if you need to install the packages

# !pip install pandas matplotlib seaborn colorama numpy requests 

# Import standard libraries
import random
import copy
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import widgets for interactivity
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown

# Import custom modules
from agents.insurer import Insurer
from agents.lst_maximalist import LstMaximalist
from agents.ct_long_term import CTLongTermAgent
from agents.ct_speculation import CTShortTermAgent
from agents.ds_long_term import DSLongTermAgent
from agents.ds_speculation import DSShortTermAgent
from agents.redemption_arbitrage import RedemptionArbitrageAgent
from agents.repurchase_arbitrage import RepurchaseArbitrageAgent
from agents.lv_depositor import LVDepositorAgent
from agents.looping import LoopingAgent
from simulator.blockchain import Blockchain
from simulator.amm import UniswapV2AMM, YieldSpaceAMM

# Set plotting style
sns.set_style('whitegrid')

# Enable inline plotting for Jupyter
%matplotlib inline

### Defining Configuration Parameters

In this section, we'll define the simulation parameters. You can adjust these parameters to see how they affect the simulation.


In [2]:
# Simulation parameters
NUM_BLOCKS = 300  # Number of blocks to simulate
INITIAL_ETH_BALANCE = 100.0  # Initial ETH balance for each agent
PSM_EXPIRY_AFTER_BLOCK = 300  # Block after which the Peg Stability Module (PSM) expires

# Token parameters
TOKEN_NAME = 'stETH'  # Name of the token to simulate
INITIAL_AGENT_TOKEN_BALANCE = 100.0  # Initial token balance for each agent
AMM_RESERVE_ETH = 1000000.0  # Initial ETH reserve in the AMM
AMM_RESERVE_TOKEN = 1000000.0  # Initial token reserve in the AMM
AMM_FEE = 0.02  # Fee percentage in the AMM, 0.02 = 2%
INITIAL_YIELD_PER_BLOCK = 0.03 / 365  # Yield per block (assuming 3% annual yield)
PSM_REDEMPTION_FEES = 0.001  # Redemption fees for the Peg Stability Module, 0.001 = 0.1%
PSM_REPURCHASE_FEES = 0.05  # Reurchase fees for the Peg Stability Module, 0.05 = 5%

# Agents to include in the simulation
AGENT_NAMES = [
    #'LstMaximalist',
    #'Insurer',
    'DSShortTerm',
    'CTShortTerm',
    #'DSLongTerm',
    #'CTLongTerm',
    'RedemptionArbitrage',
    'RepurchaseArbitrage',
    #'LVDepositor',
    'LoopingAgent'
    ]

## Initializing the Blockchain

Now we'll create the blockchain instance using the parameters defined above.


In [3]:
# Create the blockchain
chain = Blockchain(
    num_blocks=NUM_BLOCKS,
    initial_eth_balance=INITIAL_ETH_BALANCE,
    psm_expiry_after_block=PSM_EXPIRY_AFTER_BLOCK
)

## Adding Tokens and Agents

### Adding the Token

We'll add a token to the blockchain with its associated AMM.


In [4]:
# Add the token with the specified AMM
chain.add_token(
    token=TOKEN_NAME,
    risk=0.02,
    initial_agent_balance=INITIAL_AGENT_TOKEN_BALANCE,
    amm=UniswapV2AMM(
        token_symbol=TOKEN_NAME,
        reserve_eth=AMM_RESERVE_ETH,
        reserve_token=AMM_RESERVE_TOKEN,
        fee=AMM_FEE
    ),
    initial_yield_per_block=INITIAL_YIELD_PER_BLOCK
)

### Adding Agents

We'll instantiate the agents and add them to the blockchain.


In [5]:
# Instantiate agents based on AGENT_NAMES
agents = []
for name in AGENT_NAMES:
    #if name == 'LstMaximalist':
    #    agents.append(LstMaximalist(TOKEN_NAME))
    #elif name == 'Insurer':
    #    agents.append(Insurer(TOKEN_NAME))
    if name == 'DSShortTerm':
        agents.append(DSShortTermAgent(name="DS Short Term", token_symbol=TOKEN_NAME, threshold=0.01))
    elif name == 'CTShortTerm':
        agents.append(CTShortTermAgent(name="CT Short Term", token_symbol=TOKEN_NAME, buying_pressure=10))
    elif name == 'DSLongTerm':  
        agents.append(DSLongTermAgent(name="DS Long Term", token_symbol=TOKEN_NAME, buying_pressure=1))
    elif name == 'CTLongTerm':
        agents.append(CTLongTermAgent(name="CT Long Term", token_symbol=TOKEN_NAME, percentage_threshold=0.01))
    elif name == 'RedemptionArbitrage':
        agents.append(RedemptionArbitrageAgent(name="Redemption Arb", token_symbol=TOKEN_NAME))
    elif name == 'RepurchaseArbitrage':
        agents.append(RepurchaseArbitrageAgent(name="Repurchase Arb", token_symbol=TOKEN_NAME))
    elif name == 'LVDepositor':
        agents.append(LVDepositorAgent(name="LV Depositor", token_symbol=TOKEN_NAME, expected_apy=0.05))
    elif name == 'LoopingAgent':
        agents.append(LoopingAgent(
            name="Looping Agent", 
            token_symbol=TOKEN_NAME,
            initial_borrow_rate=0.001, 
            borrow_rate_changes={}, 
            max_ltv=0.7, 
            lltv=0.915))

# Add agents to the blockchain
chain.add_agents(*agents)


## Running the Simulation

Now we'll start the mining process, which runs the simulation over the specified number of blocks.


In [6]:
# Start mining without printing stats to minimize output
chain.start_mining(print_stats=False)

## Analyzing Results

After the simulation, we can analyze the collected statistics and visualize them using charts.


In [7]:
# Access stats dataframes
agents_stats = chain.stats['agents']
tokens_stats = chain.stats['tokens']
vaults_stats = chain.stats['vaults']
amms_stats = chain.stats['amms']
borrowed_eth_stats = chain.stats['borrowed_eth']
borrowed_tokens_stats = chain.stats['borrowed_tokens']


In [8]:
vaults_stats

,block,token,lp_token_price_eth,eth_balance,ds_balance_eth
0,0,stETH,0.0,0.000000,0.0
1,1,stETH,0.0,9.388918,0.0
2,2,stETH,0.0,10.349022,0.0
3,3,stETH,0.0,10.455699,0.0
4,4,stETH,0.0,10.455699,0.0
...,...,...,...,...,...
296,296,stETH,0.0,542.321358,0.0
297,297,stETH,0.0,542.321358,0.0
298,298,stETH,0.0,542.321358,0.0
299,299,stETH,0.0,542.321358,0.0


## APY

we gather all fees & earnings collected

In [9]:
sum(chain.get_amm("DS_stETH").fee_accumulated_eth)


0

In [10]:
sum(chain.get_amm("DS_stETH").fee_accumulated_token)


565

In [11]:
sum(chain.get_amm("CT_stETH").fee_accumulated_eth)


0

In [12]:
sum(chain.get_amm("CT_stETH").fee_accumulated_token)


6375

In [13]:
PSM = chain.get_psm("stETH")
PSM.total_redemption_fee
PSM.total_repurchase_fee

0.0

### APY in one variable

In [14]:
apy_total = (
    sum(chain.get_amm("DS_stETH").fee_accumulated_eth) + 
    sum(chain.get_amm("DS_stETH").fee_accumulated_token) +
    sum(chain.get_amm("CT_stETH").fee_accumulated_eth) +
    sum(chain.get_amm("CT_stETH").fee_accumulated_token) +
    PSM.total_redemption_fee +
    PSM.total_repurchase_fee
)

Note: Buying DS is currently implemented always in this way:

   "Buy DS tokens via the vault by borrowing ETH, acquiring CT/DS via the PSM,
   selling CT for ETH, and returning the remainder DS tokens to the investor"

Therefore no direct PSM/vault DS reserve selling (which increases APY) is happening and logged.

# Trading Volumes

All trades are done by agents so we best collect volumes from their trading logs.

This code shows how to get a simple tabular overview of the trades. "Action" will show regular buy/sell at vault as well as repurchase and redemption activity. 

In [15]:
all_trades = pd.DataFrame(chain.all_trades)
all_trades.groupby(["block", "token", "action"])["volume"].sum()

block  token  action
1      DS     buy       88.00000
       Token  buy       88.00000
2      DS     buy        9.00000
       Token  buy        9.00000
3      DS     buy        1.00000
       Token  buy        1.00000
279    CT     sell       0.00000
       DS     buy       51.15602
280    CT     sell       0.00000
       DS     buy       48.84398
281    CT     sell       0.00000
282    CT     sell       0.00000
283    CT     sell       0.00000
284    CT     sell       0.00000
285    CT     sell       0.00000
286    CT     sell       0.00000
287    CT     sell       0.00000
288    CT     sell       0.00000
289    CT     sell       0.00000
290    CT     sell       0.00000
291    CT     sell       0.00000
292    CT     sell       0.00000
293    CT     sell       0.00000
294    CT     sell       0.00000
295    CT     sell       0.00000
296    CT     sell       0.00000
297    CT     sell       0.00000
298    CT     sell       0.00000
299    CT     sell       0.00000
300    CT     sell    

In [16]:
all_trades = pd.DataFrame(chain.all_trades)
all_trades.groupby(["agent", "token", "action"])["volume"].sum()

agent          token  action
CT Short Term  CT     sell        0.0
DS Short Term  DS     buy       100.0
Looping Agent  DS     buy        98.0
               Token  buy        98.0
Name: volume, dtype: float64

# Looping Trades

In [ ]:
current_agent = "Looping Agent"

(
    pd.concat([
        all_trades.query("agent == @current_agent").drop(columns=["additional_info"]),
        all_trades.query("agent == @current_agent")["additional_info"].apply(pd.Series),
    ], axis=1
    )
)

,block,agent,token,volume,action,reason,ds_price,total_yield,borrow_rate
0,1,Looping Agent,Token,88.0,buy,ds_price < (total_yield - self.borrow_rate),0.019933,0.024575,0.001
1,1,Looping Agent,DS,88.0,buy,ds_price < (total_yield - self.borrow_rate),0.019933,0.024575,0.001
2,2,Looping Agent,Token,9.0,buy,ds_price < (total_yield - self.borrow_rate),0.019933,0.024493,0.001
3,2,Looping Agent,DS,9.0,buy,ds_price < (total_yield - self.borrow_rate),0.019933,0.024493,0.001
4,3,Looping Agent,Token,1.0,buy,ds_price < (total_yield - self.borrow_rate),0.019933,0.024411,0.001
5,3,Looping Agent,DS,1.0,buy,ds_price < (total_yield - self.borrow_rate),0.019933,0.024411,0.001
